In [1]:
! hostname

c01


In [ ]:
import os
import sys
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
import torch
from anndata import AnnData
import anndata
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad

from sklearn import metrics
import multiprocessing as mp
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import scipy.sparse as sp
import scipy.linalg

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import scanpy as sc
import os
import random

def format_gene_names(genes):
    return {gene.upper() if len(gene) == 1 else gene[0].upper() + gene[1:].lower() for gene in genes}

atlas_dirs = [
    '/share/home/liangzhongming/phd_code/530/CellNiche/data/Atlas1',
    '/share/home/liangzhongming/phd_code/530/CellNiche/data/Atlas2',
    '/share/home/liangzhongming/phd_code/530/CellNiche/data/Atlas3',
    '/share/home/liangzhongming/phd_code/530/CellNiche/data/Atlas4'
]
target_files = {
    'well11_cellTypt.h5ad', 
    'C57BL6J-638850.38.h5ad', 
    'Zhuang-ABCA-1.079.h5ad', 
    'S2R1_cellTypt.h5ad'
}

selected_files = {}
for atlas_dir in atlas_dirs:
    h5ad_files = [f for f in os.listdir(atlas_dir) if f.endswith('.h5ad')]
    
#     # way1: sampled
#     if len(h5ad_files) <= 10:
#         selected_files[atlas_dir] = h5ad_files
#     else:
#         selected_files[atlas_dir] = random.sample(h5ad_files, 300)
    
    # way2: all
    selected_files[atlas_dir] = h5ad_files
    
#     # way3: 4 slice
#     matched = [f for f in h5ad_files if os.path.basename(f) in target_files]
#     if matched:
#         selected_files[atlas_dir] = matched
    
print(selected_files.items())

adata_list = []
section_ids = []

for atlas_dir, files in selected_files.items():
    for file in files:
        sample = sc.read(os.path.join(atlas_dir, file))
        if 'Atlas1' in atlas_dir:
            sample.var_names = format_gene_names(sample.var_names)
            
        sample.obs.rename(columns={
            'class_name': 'class',
            'subclass_name': 'subclass',
            'supertype_name': 'supertype',
            'cluster_name': 'cluster',
            'center_x': 'x', 
            'center_y': 'y', 
            'X': 'x', 
            'Y': 'y', 
            'Z': 'z'
        }, inplace=True)
        
        if 'supertype_bootstrapping_probability' in sample.obs.columns and 'subclass_bootstrapping_probability' in sample.obs.columns:
            sample = sample[sample.obs['supertype_bootstrapping_probability'] >= 0.5]
            sample = sample[sample.obs['subclass_bootstrapping_probability'] >= 0.5]
        
        if 'spatial' not in sample.obsm:
            sample.obsm['spatial'] = sample.obs[['x', 'y']].astype(float).to_numpy()
        
        sample.obs['atlas'] = atlas_dir.split('/')[-1]
        sample.obs['slice_id'] = file.replace('.h5ad', '')
        
        adata_list.append(sample)
        section_ids.append(f"{atlas_dir.split('/')[-1]}_{file.replace('.h5ad', '')}")

adata_concat = sc.concat(
    adata_list, 
    label="sample",
    keys=section_ids,
    join='outer', # inner
    index_unique='-' 
)